In [81]:
!pip3 install glob2

In [ ]:
import pandas as pd
import os
from glob2 import glob

In [12]:
customers = pd.read_csv("/Users/sdevapalli/Downloads/Revolve/input_data/starter/customers.csv")
customers.head()

,customer_id,loyalty_score
0,C1,7
1,C2,4
2,C3,8
3,C4,5
4,C5,7


In [13]:
products = pd.read_csv("/Users/sdevapalli/Downloads/Revolve/input_data/starter/products.csv")
products.head()

,product_id,product_description,product_category
0,P01,detergent,house
1,P02,kitchen roll,house
2,P03,bin liners,house
3,P04,shower gel,house
4,P05,scented candles,house


In [97]:
# Obtain the dictionary data
path_to_json = '/Users/sdevapalli/Downloads/Revolve/input_data/starter/transactions'
# json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

dfList = []
jsonFiles = glob(f'{path_to_json}/**/*.json') #Can be used absolute or relative paths

for jsonFile in jsonFiles:
    df = pd.read_json(jsonFile, lines = True)
    dfList.append(df)
    
dfTrainingDF = pd.concat(dfList, axis=0)
cust_id_data = pd.concat([pd.DataFrame(x) for x in dfTrainingDF['basket']], keys=dfTrainingDF['customer_id']).reset_index(level=1, drop=True).reset_index()
purchase_date_data = pd.concat([pd.DataFrame(x) for x in dfTrainingDF['basket']], keys=dfTrainingDF['date_of_purchase']).reset_index(level=1, drop=True).reset_index()
transactions_data = cust_id_data.merge(purchase_date_data, on = ["product_id", "price"])
transactions_data.head()

,customer_id,product_id,price,date_of_purchase
0,C3,P18,1254,2018-12-10 22:08:00
1,C5,P44,1472,2018-12-10 16:05:00
2,C12,P58,898,2018-12-10 07:42:00
3,C12,P41,597,2018-12-10 07:42:00
4,C12,P41,597,2019-02-26 11:40:00


In [86]:
cust_trans_data = customers.merge(transactions_data, on="customer_id")
data = cust_trans_data.merge(products, on = "product_id")
data.shape

(4814, 7)

In [96]:
final_data = data.groupby(["customer_id", "loyalty_score", "product_id", "product_category"])["date_of_purchase"].nunique().reset_index().rename(columns={"date_of_purchase": "purchase_count"})

In [98]:
final_data.columns

Index(['customer_id', 'loyalty_score', 'product_id', 'product_category',
       'purchase_count'],
      dtype='object')